In [ ]:

%%capture
!pip install -q trl transformers accelerate peft datasets bitsandbytes einops loralib

In [ ]:
# import dependencies
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
import torch
import peft
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, truncation=True, padding='max_length', max_length=512)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

#model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
PEFT_MODEL = "pebbs/Falc-7b"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model_ = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model_ = PeftModel.from_pretrained(model_, PEFT_MODEL)


adapter_config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/9.45M [00:00<?, ?B/s]

In [ ]:

generation_config = model_.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
: Is a prior diagnosis of breast cancer a risk factor for breast cancer in BRCA1 and BRCA2 carriers?
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model_.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

outp = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("User Query: ", prompt.strip(': ')[0].lstrip(''))
print("Bot Response: ", outp.strip(': ')[1].rstrip('User'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


User Query:  I
Bot Response:  
CPU times: user 19.8 s, sys: 790 ms, total: 20.6 s
Wall time: 26.4 s


In [ ]:
questions = ['Is human papilloma virus associated with breast cancer?',
             'Do polymorphisms in ER-alpha gene interact with estrogen receptor status in breast cancer survival?',
             'Does lipofilling of the Breast Increase the Risk of Recurrence of Breast Cancer?',
             'Is metabolic syndrome an independent risk factor for breast cancer?']


In [ ]:
%%capture
!pip install gradio

In [ ]:
import gradio as gr

# Function to generate responses
def generate_response(user_input):
    prompt = f"""
    : {user_input}
    :
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model_.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def chat(user_input, history):
    bot_response = generate_response(user_input)
    history.append((user_input, bot_response))
    return bot_response.split(user_input)[1]


gr.ChatInterface(
    chat,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="BCDA-falcon-7b",
    description="Ask any question",
    theme="soft",
    examples=["Is human papilloma virus associated with breast cancer?"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

# Create the Gradio interface
# interface = gr.Interface(
#     fn=chat,
#     inputs=[gr.Textbox(lines=2, placeholder="Enter your message here..."), "state"],
#     outputs=["chatbot", "state"],
#     live=True
# )
# interface.launch(share=True)

Caching examples at: '/content/gradio_cached_examples/15'
Caching example 1/1


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f72d58976eb1c43369.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
